In [ ]:
import datetime as dt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from mlxtend.frequent_patterns import apriori, association_rules
import numpy as np

In [ ]:
df= pd.read_excel("online_retail_II.xlsx",sheet_name="Year 2010-2011")

In [ ]:
df.head()

In [ ]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    # dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit


In [ ]:
# Boş değerlerin doldurulması
df.dropna(axis=0, inplace=True)

# Fatura numarısında C bulunması satışın iptal edildiği anlamına gelmekte
# C içerenleri veri setinden çıkartıyoruz
df = df[~df["Invoice"].str.contains("C", na=False)]

# Ürün adedi 0'dan büyük olanları veri setinde bırakıyoruz
df = df[df["Quantity"] > 0]

# aykırı değerleri baskılama
replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")

# Ürün miktarı ve fiyatı çarparak toplam ödenen miktara ait yeni bir değişken oluşturuyoruz
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [ ]:
df.head()

In [7]:
# Veri setindeki maximum tarihten 2 gün sanrasını today_date olarak alıyoruz
today_date = dt.datetime(2011, 12, 11)

# Her bir müşteri için recency, frequency, tenure ve frekans değerlerini içeren dataframe'i oluşturuyoruz
df_cltv = df.groupby('Customer ID').agg({'InvoiceDate': [lambda date: (date.max()-date.min()).days,
                                                            lambda date: (today_date - date.min()).days],
                                            'Invoice': lambda num: num.nunique(),
                                            'TotalPrice': lambda TotalPrice: TotalPrice.sum()})
df_cltv.columns = df_cltv.columns.droplevel(0)
df_cltv.columns = ['recency', 'T', 'frequency', 'monetary']

# cltv hesaplamak için kişinin ortalama monetary değerine ihtiyacımız var
df_cltv["monetary"] = df_cltv["monetary"] / df_cltv["frequency"]
df_cltv.rename(columns={"monetary": "monetary_avg"}, inplace=True)

# recency filtre (daha saglıklı cltvp hesabı için)
df_cltv = df_cltv[(df_cltv['frequency'] > 1)]

df_cltv.head()

,recency,T,frequency,monetary_avg
Customer ID,,,,
12347.0,365,368,7,615.714286
12348.0,282,359,4,442.695000
12352.0,260,297,8,219.542500
12356.0,302,326,3,937.143333
12358.0,149,151,2,575.210000


In [8]:
# BGNBD
bgf = BetaGeoFitter(penalizer_coef=0.01)
bgf.fit(df_cltv['frequency'],
        df_cltv['recency'],
        df_cltv['T'])

# GG
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(df_cltv['frequency'], df_cltv['monetary_avg'])

# 6 aylık müşteri değeri tahmini
cltv = ggf.customer_lifetime_value(bgf,
                                    df_cltv['frequency'],
                                    df_cltv['recency'],
                                    df_cltv['T'],
                                    df_cltv['monetary_avg'],
                                    time=6,
                                    freq="D",
                                    discount_rate=0.01)
df_cltv["cltv_p"] = cltv

# 1 ile 100 arasında scale etme
scaler = MinMaxScaler(feature_range=(1, 100))
scaler.fit(df_cltv[["cltv_p"]])
df_cltv["cltv_p"] = scaler.transform(df_cltv[["cltv_p"]])

# cltv değerlerine göre segmentleme
df_cltv["cltv_p_segment"] = pd.qcut(df_cltv["cltv_p"], 3, labels=["C", "B", "A"])

df_cltv.head()

,recency,T,frequency,monetary_avg,cltv_p,cltv_p_segment
Customer ID,,,,,,
12347.0,365,368,7,615.714286,2.933545,A
12348.0,282,359,4,442.695000,1.919418,B
12352.0,260,297,8,219.542500,1.904177,B
12356.0,302,326,3,937.143333,2.802576,A
12358.0,149,151,2,575.210000,2.613230,A


In [9]:
# Birliktelik kurallarını oluşturabilmek için her işlemde alınan ve alınmayan ürünleri gösteren dataframe'i oluşturacak yardımcı fonksiyon
def create_invoice_product_df(dataframe):
    return dataframe.groupby(['Invoice', 'StockCode'])['Quantity'].sum().unstack().fillna(0). \
        applymap(lambda x: 1 if x > 0 else 0)

In [10]:
# A segmentindeki kişiler için kişisel tavsiyelerde bulunacağız.
# Filtreleme yapalım
a_segment_ids = df_cltv[df_cltv["cltv_p_segment"] == "A"].index # A segmentindeki kişilerin id'leri
a_segment_df = df[df["Customer ID"].isin(a_segment_ids)]# A segmentindeki kişilerin kümalatif hale getirilmemiş dataframe'i(ilk df'den) 
invoice_product_df = create_invoice_product_df(a_segment_df)
invoice_product_df.head()

StockCode,10002,10080,10120,10125,10133,10135,11001,15030,15034,15036,...,90214R,90214S,90214V,90214Y,BANK CHARGES,C2,DOT,M,PADS,POST
Invoice,,,,,,,,,,,,,,,,,,,,,
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536370,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
536376,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Birliktelik kurallarının oluşturulması
frequent_itemsets = apriori(invoice_product_df, min_support=0.005, use_colnames=True, low_memory=True)
rules_segment_A = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
rules_segment_A = rules_segment_A.sort_values("confidence", ascending=False)
rules_segment_A.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1029,(22918),(22917),0.010814,0.011549,0.010499,0.970874,84.068844,0.010374,33.936833
1032,(22920),(22917),0.011129,0.011549,0.010709,0.962264,83.323328,0.010580,26.193963
1024,(22916),(22917),0.010814,0.011549,0.010394,0.961165,83.228155,0.010269,25.452625
2494,"(23171, 22699)",(23170),0.011339,0.024567,0.010709,0.944444,38.443732,0.010430,17.557795
1027,(22916),(22920),0.010814,0.011129,0.010184,0.941748,84.624015,0.010063,16.975626


In [12]:
# antecedents ve consequents değişkenleri frozenset bunları listes haline çevirelim
rules_segment_A["consequents"] = rules_segment_A["consequents"].apply(lambda x: list(x)).astype(str)
rules_segment_A["antecedents"] = rules_segment_A["antecedents"].apply(lambda x: list(x)).astype(str)

In [13]:
# A segmentinden bir id seçip özelleştirilmiş ürün önerisini verelim daha sonra bunu fonskiyonlaştırıp tüm müşteriler için yapıcaz
a_segment_df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
9,536368,22960,JAM MAKING SET WITH JARS,6.0,2010-12-01 08:34:00,4.25,13047.0,United Kingdom,25.50
10,536368,22913,RED COAT RACK PARIS FASHION,3.0,2010-12-01 08:34:00,4.95,13047.0,United Kingdom,14.85
11,536368,22912,YELLOW COAT RACK PARIS FASHION,3.0,2010-12-01 08:34:00,4.95,13047.0,United Kingdom,14.85
12,536368,22914,BLUE COAT RACK PARIS FASHION,3.0,2010-12-01 08:34:00,4.95,13047.0,United Kingdom,14.85
13,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32.0,2010-12-01 08:34:00,1.69,13047.0,United Kingdom,54.08


In [14]:
# customer id'si 12347.00 olan müşterinin en son yaptığı alışveriş tarhindeki ürünleri quantity ve total price'a göre sort ettik
# last_invoice_products dataframe'ine atadık
last_Invoice_products = a_segment_df.loc[(a_segment_df["Customer ID"]== 12347.0) & 
                                         (a_segment_df["InvoiceDate"]==a_segment_df[a_segment_df["Customer ID"]== 12347.0]
                                          ["InvoiceDate"].max())].sort_values(["Quantity","TotalPrice"], ascending=False)
last_Invoice_products

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
535009,581180,23084,RABBIT NIGHT LIGHT,24.0,2011-12-07 15:52:00,1.79,12347.0,Iceland,42.96
535008,581180,21731,RED TOADSTOOL LED NIGHT LIGHT,24.0,2011-12-07 15:52:00,1.65,12347.0,Iceland,39.60
535006,581180,21064,BOOM BOX SPEAKER BOYS,24.0,2011-12-07 15:52:00,1.25,12347.0,Iceland,30.00
535007,581180,84625A,PINK NEW BAROQUECANDLESTICK CANDLE,24.0,2011-12-07 15:52:00,0.85,12347.0,Iceland,20.40
535013,581180,23506,MINI PLAYING CARDS SPACEBOY,20.0,2011-12-07 15:52:00,0.42,12347.0,Iceland,8.40
535014,581180,23508,MINI PLAYING CARDS DOLLY GIRL,20.0,2011-12-07 15:52:00,0.42,12347.0,Iceland,8.40
535012,581180,23271,CHRISTMAS TABLE SILVER CANDLE SPIKE,16.0,2011-12-07 15:52:00,0.83,12347.0,Iceland,13.28
535011,581180,21265,PINK GOOSE FEATHER TREE 60CM,12.0,2011-12-07 15:52:00,1.95,12347.0,Iceland,23.40
535004,581180,23497,CLASSIC CHROME BICYCLE BELL,12.0,2011-12-07 15:52:00,1.45,12347.0,Iceland,17.40
535010,581180,20719,WOODLAND CHARLOTTE BAG,10.0,2011-12-07 15:52:00,0.85,12347.0,Iceland,8.50


In [39]:
# Oluşan kurrallarda bazı ürünler yer almayabilir herhangi birini bulana kadar sırayla last_Invoice_products daki ürünler denenecek
for i in range(len(last_Invoice_products)):
    product = last_Invoice_products.iloc[i, 1] # ürün
   # Bu ürünün içinde bulunduğu kurallar
    customize_recommend_df = rules_segment_A[rules_segment_A.antecedents.apply(lambda x: str(product) in x)]
    if len(customize_recommend_df) > 0:
        print(product)
        print("Recommended Product : " ,customize_recommend_df.iloc[0, 1]) # en yüksek confidence'a sahip ürün veya ürünler önerilir
        break

20719
Recommended Product :  [20724]


In [16]:
# Tüm müşteriler için segmentleri ve son alışverişlerindeki  ürünleri baz alarak ürün önerilerini yapalım

# Kuralların oluşturmak için create rules fonskiyonu oluşturuyoruz
def create_rules(dataframe):
    dataframe = create_invoice_product_df(dataframe)
    frequent_itemsets = apriori(dataframe, min_support=0.005, use_colnames=True,low_memory=True)
    rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
    rules["consequents"] = rules["consequents"].apply(lambda x: list(x)).astype(str)
    rules["antecedents"] = rules["antecedents"].apply(lambda x: list(x)).astype(str)
    return rules


# id'lerin alınması
a_segment_ids = df_cltv[df_cltv["cltv_p_segment"] == "A"].index
b_segment_ids = df_cltv[df_cltv["cltv_p_segment"] == "B"].index
c_segment_ids = df_cltv[df_cltv["cltv_p_segment"] == "C"].index

# bu id'lere göre df'lerin indirgenmesi
a_segment_df = df[df["Customer ID"].isin(a_segment_ids)]
b_segment_df = df[df["Customer ID"].isin(b_segment_ids)]
c_segment_df = df[df["Customer ID"].isin(c_segment_ids)]

# Taviye edilecek ürünleri göstermek için df_cltv dataframme'ine recommended products değişkeni ekleyelim
df_cltv["Recommended Product"] = None

In [17]:
def add_recommended_products(dataframe, customer_ids):
    rules = create_rules(dataframe)
    rules = rules.sort_values("confidence",ascending=False)
   
    
    for id in customer_ids:
        last_Invoice_products = dataframe.loc[(dataframe["Customer ID"]== id) & 
                                          (dataframe["InvoiceDate"]==dataframe[dataframe["Customer ID"]== id]
                                           ["InvoiceDate"].max())].sort_values(["Quantity","TotalPrice"], ascending=False)
        
        for i in range(len(last_Invoice_products)):
            product = last_Invoice_products.iloc[i, 1]
            customize_recommend_df = rules[rules.antecedents.apply(lambda x: str(product) in x)]
            if len(customize_recommend_df) > 0:
                df_cltv.loc[df_cltv.index == id,"Recommended Product"] = customize_recommend_df.iloc[0, 1]
                break

In [18]:
add_recommended_products(a_segment_df,a_segment_ids)
add_recommended_products(b_segment_df,b_segment_ids)
add_recommended_products(c_segment_df,c_segment_ids)

In [19]:
df_cltv.head(10)

,recency,T,frequency,monetary_avg,cltv_p,cltv_p_segment,Recommended Product
Customer ID,,,,,,,
12347.0,365,368,7,615.714286,2.933545,A,[20724]
12348.0,282,359,4,442.695000,1.919418,B,None
12352.0,260,297,8,219.542500,1.904177,B,None
12356.0,302,326,3,937.143333,2.802576,A,[22699]
12358.0,149,151,2,575.210000,2.613230,A,['15056BL']
12359.0,274,332,4,1371.850000,4.065195,A,[84879]
12360.0,148,201,3,878.533333,3.401064,A,[22328]
12362.0,292,296,10,522.623000,3.621753,A,[23266]
12363.0,132,243,2,276.000000,1.490470,C,None


In [20]:
# Bazı müşteriler için öneri ürün çıkmamış
# Bunun sebebi son yapılan alışverişte alınan ürünlerin kuralların içinde bulunmaması
# Çözüm olarak daha önceki alışverişlerine bakabiliriz ama  segmente göre en çok satılan ürünü önericem
df_cltv["Recommended Product"].fillna(np.NaN, inplace=True)

a_rules = create_rules(a_segment_df)
a_product = a_rules.sort_values("antecedent support", ascending=False)["antecedents"][0]
df_cltv.loc[(df_cltv["cltv_p_segment"]=="A")& (df_cltv["Recommended Product"].isnull()),"Recommended Product"] = a_product

b_rules = create_rules(b_segment_df)
b_product = b_rules.sort_values("antecedent support", ascending=False)["antecedents"][0]
df_cltv.loc[(df_cltv["cltv_p_segment"]=="B")& (df_cltv["Recommended Product"].isnull()),"Recommended Product"] = b_product

c_rules = create_rules(c_segment_df)
c_product = c_rules.sort_values("antecedent support", ascending=False)["antecedents"][0]
df_cltv.loc[(df_cltv["cltv_p_segment"]=="C")& (df_cltv["Recommended Product"].isnull()),"Recommended Product"] = c_product

In [21]:
df_cltv.head()

,recency,T,frequency,monetary_avg,cltv_p,cltv_p_segment,Recommended Product
Customer ID,,,,,,,
12347.0,365,368,7,615.714286,2.933545,A,[20724]
12348.0,282,359,4,442.695000,1.919418,B,[20723]
12352.0,260,297,8,219.542500,1.904177,B,[20723]
12356.0,302,326,3,937.143333,2.802576,A,[22699]
12358.0,149,151,2,575.210000,2.613230,A,['15056BL']


In [24]:
# Recommended Products değişkeni ürünlerin kodlarını tutmakta, aşağıdaki fonksiyon ile ürünlerin isimlerine ulaşabiliriz.
def check_id(stock_code):
    product_name = df[df["StockCode"] == stock_code][["Description"]].values[0].tolist()
    return print(product_name)

check_id(20724)

['RED RETROSPOT CHARLOTTE BAG']
